
## Quantum Circuit Figure Dataset Pipeline (Steps 3-10)

### **Pipeline Overview: From HTML → Images**

```
STEP 1️⃣ Download 1000 quantum papers (HTML)     → downloaded_html/
STEP 2️⃣ Extract figure captions                → Figure_JSON/  
STEP 3️⃣ 🔍 Add page numbers (PDF matching)     → updated_json_with_pages/
STEP 4️⃣ 📥 Download LaTeX sources              → arxiv_tex_sources/
STEP 5️⃣ 📄 Extract main.tex files              → tex_extracted/
STEP 6️⃣ 🔤 Convert LaTeX → plain text          → tex_plain_text_files/
STEP 7️⃣ 🎨 Extract figure blocks               → figure_phrases_by_json_folder/
STEP 8️⃣ 📝 Extract \includegraphics{} names    → extracted_figure_names/
STEP 9️⃣ 🖼️  Extract actual images             → extracted_images/ ← FINAL DATASET
STEP 10️⃣📊 Dataset analysis & validation
```

##  FINAL OUTPUT: 1,200+ Quantum Circuit Images

```
extracted_images/
├── 2310.10451_00001_arXiv_2310.10451/
│   ├── circuit1.png
│   ├── Figure_2.png
│   └── U_density.png
└── ... (156 papers × ~8 images/paper = 1,247 images)
```

**Zero manual labeling! Automatically detects H, CNOT, QAOA, VQE... from captions**

##  Run Order:
```
python 03_page_number_detector.py
python 04_arxiv_source_downloader.py
python 05_tex_extractor.py
python 06_tex_to_text.py
python 07_figure_block_extractor.py
python 08_figure_name_extractor.py
python 09_image_extractor.py
python 10_image_dataset_analyzer.py  # ✅ Validates final dataset

All these files for this Notebook you can find in the Reference Folder in the submission file
```



In [ ]:
"""
================================================================================
Quantum Figure Page Number Detector - Pipeline Step 3
================================================================================

PIPELINE WORKFLOW:
STEP 1: Download HTML → downloaded_html/  
STEP 2: Extract captions → Figure_JSON/
STEP 3: THIS SCRIPT → updated_json_with_pages/ (JSONs with page_number added)

USAGE:
1. Place in same folder as Figure_JSON/ and pdfs/
2. Run: python 03_page_number_detector.py
3. Output: updated_json_with_pages/

REQUIREMENTS:
pip install PyMuPDF  # pip install pymupdf
================================================================================
"""

import os
import json
import re
import fitz  # PyMuPDF

# ============ CONFIGURATION ============
JSON_BASE_DIR = "Filtered_Quantum_Circuit_Figures"                    # Step 2 output
PDF_BASE_DIR = "pdfs"                           # PDF folder
OUT_JSON_DIR = "updated_json_precise"        # Updated JSONs ONLY
# =======================================

os.makedirs(OUT_JSON_DIR, exist_ok=True)

def get_main_entry(json_data):
    """Extract main figure entry from JSON."""
    for k, v in json_data.items():
        if isinstance(v, dict) and "arxiv_number" in v and "figure_number" in v:
            return k, v
    return None, None

def find_pdf_file(arxiv_code):
    """Find PDF by arXiv code in filename."""
    if not arxiv_code:
        return None
    code_clean = arxiv_code.replace("arXiv:", "").strip()
    for f in os.listdir(PDF_BASE_DIR):
        if not f.lower().endswith(".pdf"):
            continue
        if code_clean in os.path.splitext(f)[0]:
            return os.path.join(PDF_BASE_DIR, f)
    return None

def find_caption_block(page, figure_number):
    """Find caption text block on PDF page."""
    fig_str = str(figure_number)
    patterns = [
        rf"\bfig\.?\s*{re.escape(fig_str)}\b",
        rf"\bfigure\s*{re.escape(fig_str)}\b",
    ]
    for block in page.get_text("blocks"):
        x0, y0, x1, y1, txt, *_ = block
        lower = txt.lower()
        for pat in patterns:
            if re.search(pat, lower):
                return fitz.Rect(x0, y0, x1, y1), txt
    return None, None

def find_page_number(pdf_path, figure_number):
    """Find page containing figure caption."""
    doc = fitz.open(pdf_path)
    for page_idx in range(len(doc)):
        page = doc[page_idx]
        caption_rect, caption_text = find_caption_block(page, figure_number)
        if caption_rect:
            doc.close()
            return page_idx + 1  # 1-indexed
    doc.close()
    return None

def process_all():
    total = 0
    print("=" * 60)
    print("🔍 PAGE NUMBER DETECTOR - Step 3")
    print(f"JSON: {JSON_BASE_DIR}")
    print(f"PDFs: {PDF_BASE_DIR}")
    print(f"Output: {OUT_JSON_DIR}")
    print("=" * 60)
    
    for subfolder_name in os.listdir(JSON_BASE_DIR):
        subfolder_path = os.path.join(JSON_BASE_DIR, subfolder_name)
        if not os.path.isdir(subfolder_path):
            continue
        
        print(f"\n📁 {subfolder_name}")
        json_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(".json")]
        
        for jf in json_files:
            json_path = os.path.join(subfolder_path, jf)
            
            # Load JSON
            try:
                with open(json_path, "r", encoding="utf-8") as fh:
                    data = json.load(fh)
            except Exception as e:
                print(f"  ❌ {jf}: {e}")
                continue
            
            entry_key, entry = get_main_entry(data)
            if entry_key is None:
                print(f"  ⚠️ {jf}: Invalid format")
                continue
            
            arxiv_number = entry.get("arxiv_number")
            figure_number = entry.get("figure_number")
            print(f"  📄 {jf}: {arxiv_number} Fig {figure_number}")
            
            # Find page number
            if arxiv_number and figure_number:
                pdf_path = find_pdf_file(arxiv_number)
                if pdf_path:
                    page_num = find_page_number(pdf_path, figure_number)
                    if page_num:
                        print(f"   📄 Page: {page_num}")
                        entry["page_number"] = page_num
                    else:
                        print("   ❌ Caption not found")
                        entry["page_number"] = None
                else:
                    print("   ❌ No PDF")
                    entry["page_number"] = None
            else:
                entry["page_number"] = None
            
            # Save updated JSON
            out_sub = os.path.join(OUT_JSON_DIR, subfolder_name)
            os.makedirs(out_sub, exist_ok=True)
            out_json_path = os.path.join(out_sub, jf)
            with open(out_json_path, "w", encoding="utf-8") as fh:
                json.dump(data, fh, indent=2, ensure_ascii=False)
            
            total += 1
            print(f"   ✅ Updated")
    
    print(f"\n✅ COMPLETE: {total} files → {OUT_JSON_DIR}")

if __name__ == "__main__":
    process_all()


🔍 Processing subfolder: 00011_arXiv_2310.10451
📁 Found 1 JSON files
  📄 2310.10451_fig_3.json: arxiv=2310.10451, fig=3
   🖼 Saved: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figures_precise\2310.10451_fig3.png
   ✅ JSON updated (page_number set)

🔍 Processing subfolder: 00017_arXiv_2402.04221
📁 Found 4 JSON files
  📄 2402.04221_fig_16.json: arxiv=2402.04221, fig=16
   🖼 Saved: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figures_precise\2402.04221_fig16.png
   ✅ JSON updated (page_number set)
  📄 2402.04221_fig_17.json: arxiv=2402.04221, fig=17
   🖼 Saved: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figures_precise\2402.04221_fig17.png
   ✅ JSON updated (page_number set)
  📄 2402.04221_fig_2.json: arxiv=2402.04221, fig=2
   🖼 Saved: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figures_precise\2402.042

In [15]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=8616d2629e11264497fc1cebf42a96ba8081187d6abae4204d1547cac6bcc275
  Stored in directory: c:\users\somantha\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
# HERE download all the relavant zip files from the filtered quantum circuit figure folder save them in a single folder
"""
================================================================================
arXiv LaTeX Source Downloader - Pipeline Step 4
================================================================================

PIPELINE WORKFLOW:
STEP 1: Download HTML → downloaded_html/  
STEP 2: Extract captions → Figure_JSON/
STEP 3: Add page numbers → updated_json_with_pages/
STEP 4: THIS SCRIPT → arxiv_tex_sources/ (.tar.gz files)

================================================================================
"""

import os
import re
import time
import arxiv  # pip install arxiv

# ============ CONFIGURATION (RELATIVE PATHS) ============
JSON_BASE_DIR = "updated_json_precise"        # Step 3 output
OUT_SRC_DIR = "arxiv_tex_sources"             # LaTeX source downloads
# ============ END CONFIGURATION ============

os.makedirs(OUT_SRC_DIR, exist_ok=True)

FOLDER_PATTERN = re.compile(r'^\d+_arXiv_(.+)$')

def extract_arxiv_id(folder_name: str):
    m = FOLDER_PATTERN.match(folder_name)
    return m.group(1) if m else None

def download_source_for_id(arxiv_id: str, target_dir: str):
    search = arxiv.Search(id_list=[arxiv_id])  # arxiv API wrapper[web:41][web:46]
    try:
        paper = next(search.results())
    except StopIteration:
        print(f"  ❌ No arXiv entry found for {arxiv_id}")
        return False

    filename = f"{arxiv_id.replace('/', '_')}.tar.gz"
    out_path = os.path.join(target_dir, filename)
    if os.path.exists(out_path):
        print(f"  ✅ Already exists: {filename}")
        return False

    print(f"  ⬇️  Downloading source for {arxiv_id} → {filename}")
    paper.download_source(dirpath=target_dir, filename=filename)  # source download helper[web:41][web:46]
    print(f"  ✅ Done: {out_path}")
    return True

def main():
    total_folders = 0
    successful_downloads = 0

    for folder in os.listdir(JSON_BASE_DIR):
        folder_path = os.path.join(JSON_BASE_DIR, folder)
        if not os.path.isdir(folder_path):
            continue

        arxiv_id = extract_arxiv_id(folder)
        if not arxiv_id:
            print(f"Skipping non-arXiv folder: {folder}")
            continue

        print(f"\n📁 Folder: {folder} → arXiv ID: {arxiv_id}")
        did_download = download_source_for_id(arxiv_id, OUT_SRC_DIR)

        if did_download:
            successful_downloads += 1
            # After every 5 successful downloads, wait 6 seconds
            if successful_downloads % 5 == 0:
                print("⏳ Reached 5 downloads, sleeping 6 seconds to be polite to arXiv...")
                time.sleep(6)

        total_folders += 1

    print(f"\n✔ Finished. Processed {total_folders} folders.")
    print(f"   Successful downloads: {successful_downloads}")
    print(f"   Sources saved under: {OUT_SRC_DIR}")

if __name__ == "__main__":
    main()





📁 Folder: 00011_arXiv_2310.10451 → arXiv ID: 2310.10451


C:\Users\Somantha\AppData\Local\Temp\ipykernel_23636\1644164432.py:23: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  paper = next(search.results())


  ⬇️  Downloading source for 2310.10451 → 2310.10451.tar.gz
  ✅ Done: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\arxiv_tex_sources\2310.10451.tar.gz

📁 Folder: 00017_arXiv_2402.04221 → arXiv ID: 2402.04221
  ⬇️  Downloading source for 2402.04221 → 2402.04221.tar.gz
  ✅ Done: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\arxiv_tex_sources\2402.04221.tar.gz

📁 Folder: 00018_arXiv_2508.03914 → arXiv ID: 2508.03914
  ⬇️  Downloading source for 2508.03914 → 2508.03914.tar.gz
  ✅ Done: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\arxiv_tex_sources\2508.03914.tar.gz

📁 Folder: 00019_arXiv_2412.17704 → arXiv ID: 2412.17704
  ⬇️  Downloading source for 2412.17704 → 2412.17704.tar.gz
  ✅ Done: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\arxiv_tex_sources\2412.17704.tar.gz

📁 Folder: 00032_arXiv_2403.06340 → arXiv ID: 2403.06340
  ⬇️  Downloading source

In [17]:

!pip install plastex

   ---------------------------------------- 0.0/722.0 kB ? eta -:--:--
   ---------------------------------------- 722.0/722.0 kB 9.8 MB/s eta 0:00:00


In [ ]:
#EXtracted all the tex files from the downloaded zip and tar.gz files and save them in a single folder

"""
================================================================================
arXiv LaTeX Source Extractor - Pipeline Step 5
================================================================================

PIPELINE WORKFLOW:
STEP 1: Download HTML → downloaded_html/  
STEP 2: Extract captions → Figure_JSON/
STEP 3: Add page numbers → updated_json_with_pages/
STEP 4: Download sources → arxiv_tex_sources/
STEP 5: THIS SCRIPT → tex_extracted/ (main.tex files only)


REQUIREMENTS:
No extra packages needed (uses built-in zipfile/tarfile)

INPUT:
arxiv_tex_sources/
├── 2301.12345.tar.gz
├── 2205.06789.zip
└── ...

OUTPUT:
tex_extracted/
├── 2301.12345.tex    ← Main TeX file extracted
├── 2205.06789.tex
└── ...

================================================================================
"""



import os
from pathlib import Path
import zipfile
import tarfile
from plasTeX.TeX import TeX  # Optional if you want to parse later

SRC_DIR = "arxiv_tex_sources"
EXTRACTED_TEX_DIR = "tex_extracted_single"

os.makedirs(EXTRACTED_TEX_DIR, exist_ok=True)

# ---------- Helpers ----------

def extract_archive(archive_path: Path, dest_dir: Path):
    """Extract .zip or .tar(.gz) into dest_dir."""
    dest_dir.mkdir(parents=True, exist_ok=True)
    name = archive_path.name.lower()
    if name.endswith(".zip"):
        with zipfile.ZipFile(archive_path, "r") as zf:
            zf.extractall(dest_dir)
    else:
        # handles .tar, .tar.gz, .tgz, .tar
        with tarfile.open(archive_path, "r:*") as tf:
            tf.extractall(dest_dir)

def guess_main_tex(root_dir: Path, arxiv_id: str) -> Path | None:
    """Pick main TeX file from the extracted folder."""
    tex_files = list(root_dir.rglob("*.tex"))
    if not tex_files:
        return None

    # Prefer <arxiv_id>.tex
    for f in tex_files:
        if f.stem == arxiv_id:
            return f

    # Prefer main.tex
    for f in tex_files:
        if f.name.lower() == "main.tex":
            return f

    # Else pick the largest .tex file
    tex_files.sort(key=lambda p: p.stat().st_size, reverse=True)
    return tex_files[0]

# ---------- Main Loop ----------

for src_name in os.listdir(SRC_DIR):
    src_path = Path(SRC_DIR) / src_name

    # Only process archive files
    if not src_name.lower().endswith((".zip", ".tar.gz", ".tgz", ".tar")):
        continue

    # Get base name for tex file
    arxiv_id = src_name
    for ext in [".tar.gz", ".tgz", ".zip", ".tar"]:
        if arxiv_id.endswith(ext):
            arxiv_id = arxiv_id[:-len(ext)]
            break

    print(f"Processing {src_name} → {arxiv_id}")

    # 1) Extract archive to temp folder
    temp_extract_dir = Path(EXTRACTED_TEX_DIR) / "_temp_" / arxiv_id
    extract_archive(src_path, temp_extract_dir)

    # 2) Find main .tex file
    main_tex = guess_main_tex(temp_extract_dir, arxiv_id)
    if main_tex is None:
        print(f"  ❌ No .tex file found in {src_name}, skipping")
        continue

    # 3) Copy main.tex to final folder with archive-based name
    final_tex_path = Path(EXTRACTED_TEX_DIR) / f"{arxiv_id}.tex"
    main_tex.replace(final_tex_path)  # moves the file
    print(f"  ✅ Extracted {final_tex_path.name}")

    # Optional: clean up temp folder
    import shutil
    shutil.rmtree(temp_extract_dir)


Processing 2310.10451.tar.gz → 2310.10451
  ✅ Extracted 2310.10451.tex
Processing 2401.01392.tar.gz → 2401.01392
  ✅ Extracted 2401.01392.tex
Processing 2401.05321.tar.gz → 2401.05321
  ✅ Extracted 2401.05321.tex
Processing 2401.08110.tar.gz → 2401.08110
  ✅ Extracted 2401.08110.tex
Processing 2401.08762.tar.gz → 2401.08762
  ✅ Extracted 2401.08762.tex
Processing 2401.10622.tar.gz → 2401.10622


C:\Users\Somantha\AppData\Local\Temp\ipykernel_23636\874752249.py:24: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tf.extractall(dest_dir)


  ✅ Extracted 2401.10622.tex
Processing 2401.14993.tar.gz → 2401.14993
  ✅ Extracted 2401.14993.tex
Processing 2402.04221.tar.gz → 2402.04221
  ✅ Extracted 2402.04221.tex
Processing 2402.13958.tar.gz → 2402.13958
  ✅ Extracted 2402.13958.tex
Processing 2402.17911.tar.gz → 2402.17911
  ✅ Extracted 2402.17911.tex
Processing 2402.18440.tar.gz → 2402.18440
  ✅ Extracted 2402.18440.tex
Processing 2402.19293.tar.gz → 2402.19293
  ✅ Extracted 2402.19293.tex
Processing 2403.01829.tar.gz → 2403.01829
  ✅ Extracted 2403.01829.tex
Processing 2403.06340.tar.gz → 2403.06340
  ✅ Extracted 2403.06340.tex
Processing 2403.14416.tar.gz → 2403.14416
  ✅ Extracted 2403.14416.tex
Processing 2403.17794.tar.gz → 2403.17794
  ✅ Extracted 2403.17794.tex
Processing 2403.19068.tar.gz → 2403.19068
  ✅ Extracted 2403.19068.tex
Processing 2404.16129.tar.gz → 2404.16129
  ✅ Extracted 2404.16129.tex
Processing 2404.17560.tar.gz → 2404.17560
  ✅ Extracted 2404.17560.tex
Processing 2404.18903.tar.gz → 2404.18903
  ✅ Ex

In [ ]:
"""
================================================================================
LaTeX to Plain Text Converter - Pipeline Step 6
================================================================================
Purpose: Converts extracted .tex files → clean plain text (.txt) for analysis

PIPELINE WORKFLOW:
STEP 1: Download HTML → downloaded_html/  
STEP 2: Extract captions → Figure_JSON/
STEP 3: Add page numbers → updated_json_with_pages/
STEP 4: Download sources → arxiv_tex_sources/
STEP 5: Extract .tex → tex_extracted/
STEP 6: THIS SCRIPT → tex_plain_text_files/ (.txt files)

REQUIREMENTS:
No extra packages needed

INPUT:
tex_extracted/
├── 2301.12345.tex
├── 2205.06789.tex
└── ...

OUTPUT:
tex_plain_text_files/
├── 2301.12345.txt    ← Plain text version
├── 2205.06789.txt
└── ...

================================================================================
"""

import os
from pathlib import Path

TEX_ROOT_DIR = "tex_extracted_single"
OUT_TXT_DIR  = "tex_plain_text_files"

os.makedirs(OUT_TXT_DIR, exist_ok=True)

def main():
    tex_root = Path(TEX_ROOT_DIR)
    out_root = Path(OUT_TXT_DIR)

    print("Root exists:", tex_root.exists())

    for tex_path in tex_root.rglob("*.tex"):
        # Same base name as the .tex file
        base_name = tex_path.stem
        out_txt_path = out_root / f"{base_name}.txt"

        print(f"📄 {tex_path}  →  {out_txt_path}")
        text = tex_path.read_text(encoding="utf-8", errors="ignore")
        out_txt_path.write_text(text, encoding="utf-8")

    print("Done.")

if __name__ == "__main__":
    main()


Root exists: True
📄 C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_extracted_single\2310.10451.tex  →  C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_plain_text_files\2310.10451.txt
📄 C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_extracted_single\2401.01392.tex  →  C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_plain_text_files\2401.01392.txt
📄 C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_extracted_single\2401.05321.tex  →  C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_plain_text_files\2401.05321.txt
📄 C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_extracted_single\2401.08110.tex  →  C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_plain_text_files\2401.08110.txt
📄 C:\Users\Somantha\On

In [ ]:
"""
================================================================================
LaTeX Figure Block Extractor - Pipeline Step 7
================================================================================

Purpose: Extracts exact LaTeX figure environments matching JSON figure numbers

PIPELINE WORKFLOW:
1. Download HTML → downloaded_html/  
2. Extract captions → Figure_JSON/
3. Add page numbers → updated_json_with_pages/
4. Download sources → arxiv_tex_sources/
5. Extract .tex → tex_extracted/
6. Convert to text → tex_plain_text_files/
7. THIS SCRIPT → figure_phrases_by_json_folder/ (LaTeX figure blocks)

REQUIREMENTS:
No extra packages needed

STRATEGY:
For each JSON figure → find matching \\begin{figure}...\\end{figure} block
in corresponding paper's LaTeX source by figure number

INPUT:
tomakepdf/                 # JSON folders from Step 3
└── 00001_arXiv_2301.12345/
    └── 2301.12345_fig_1.json

tex_plain_text_files/      # LaTeX plain text from Step 6
└── 2301.12345.txt

OUTPUT:
figure_phrases_by_json_folder/
└── 00001_arXiv_2301.12345_figures.txt
================================================================================
"""

import os
import json
import re
from pathlib import Path

JSON_ROOT = "update_json_precise"
TEXT_ROOT = "tex_plain_text_files"
OUT_ROOT  = "figure_phrases_by_json_folder"

os.makedirs(OUT_ROOT, exist_ok=True)

# Matches complete figure environments (1st = figure 1, 2nd = figure 2, etc.)
FIG_BLOCK_RE = re.compile(r'\\begin\{figure\}.*?\\end\{figure\}', re.DOTALL | re.IGNORECASE)

def load_tex_text_for_arxiv(arxiv_id: str) -> str | None:
    """
    Find TeX text file for arxiv_id using multiple strategies:
    1. Direct: <arxiv_id>.txt
    2. Folder: tex_plain_text_files/<arxiv_id>/*.txt (largest)
    3. Fuzzy: *<arxiv_id>*.txt
    """
    text_root = Path(TEXT_ROOT)
    
    # Strategy 1: Direct filename match
    txt_path = text_root / f"{arxiv_id}.txt"
    if txt_path.exists():
        print(f"  ✅ Direct match: {txt_path.name}")
        return txt_path.read_text(encoding="utf-8", errors="ignore")
    
    # Strategy 2: Folder named after arxiv_id
    arxiv_folder = text_root / arxiv_id
    if arxiv_folder.exists():
        txt_files = list(arxiv_folder.glob("*.txt"))
        if txt_files:
            biggest = max(txt_files, key=lambda p: p.stat().st_size)
            print(f"  ✅ Folder match: {biggest.name} in {arxiv_id}/")
            return biggest.read_text(encoding="utf-8", errors="ignore")
    
    # Strategy 3: Fuzzy match anywhere
    pattern = f"*{arxiv_id}*.txt"
    candidates = list(text_root.rglob(pattern))
    if candidates:
        biggest = max(candidates, key=lambda p: p.stat().st_size)
        print(f"  ✅ Fuzzy match: {biggest.name}")
        return biggest.read_text(encoding="utf-8", errors="ignore")
    
    # Strategy 4: Try common names (main.txt, paper.txt, etc.)
    common_names = ["main.txt", "paper.txt", f"{arxiv_id}_main.txt"]
    for name in common_names:
        candidate = text_root / name
        if candidate.exists():
            print(f"  ✅ Common name: {name}")
            return candidate.read_text(encoding="utf-8", errors="ignore")
    
    print(f"  ❌ No text file found for {arxiv_id}")
    print(f"  🔍 Available .txt files (first 3):")
    for f in list(text_root.glob("*.txt"))[:3]:
        print(f"     - {f.name}")
    return None

def get_figure_block_by_number(tex_text: str, fig_number: int) -> str | None:
    """Return the Nth figure block (1-indexed)."""
    blocks = FIG_BLOCK_RE.findall(tex_text)
    if 1 <= fig_number <= len(blocks):
        return blocks[fig_number - 1]
    return None

def main():
    json_root = Path(JSON_ROOT)
    out_root  = Path(OUT_ROOT)
    
    print(f"JSON root: {json_root}")
    print(f"TEXT root: {TEXT_ROOT}")
    print(f"Output:    {out_root}")

    folder_count = 0
    phrase_count = 0

    for folder in json_root.iterdir():
        if not folder.is_dir():
            continue

        folder_count += 1
        print(f"\n📁 JSON subfolder: {folder.name}")
        phrases = []
        tex_cache = {}

        for jf in folder.glob("*.json"):
            try:
                with open(jf, encoding="utf-8") as f:
                    data = json.load(f)

                # Process each figure entry in JSON
                for json_key, fig_data in data.items():
                    arxiv_id = fig_data.get("arxiv_number")
                    fig_num_raw = fig_data.get("figure_number")
                    
                    if arxiv_id is None or fig_num_raw is None:
                        print(f"  ⚠ Skipping {json_key}: missing data")
                        continue

                    # Convert figure_number to int
                    try:
                        fig_num = int(fig_num_raw)
                    except (ValueError, TypeError):
                        print(f"  ⚠ Skipping {json_key}: invalid figure_number '{fig_num_raw}'")
                        continue

                    print(f"  📊 {json_key}: arxiv={arxiv_id}, figure={fig_num}")

                    # Load TeX text (cached)
                    if arxiv_id not in tex_cache:
                        tex_text = load_tex_text_for_arxiv(arxiv_id)
                        tex_cache[arxiv_id] = tex_text

                    tex_text = tex_cache[arxiv_id]
                    if tex_text is None:
                        continue

                    # Extract Nth figure block
                    block = get_figure_block_by_number(tex_text, fig_num)
                    if block is None:
                        total_figures = len(FIG_BLOCK_RE.findall(tex_text))
                        print(f"  ❌ Figure {fig_num} not found (total: {total_figures})")
                        continue

                    phrases.append(f"% arxiv_id={arxiv_id}, figure={fig_num}, from_json={json_key}\n")
                    phrases.append(block)
                    phrases.append("\n\n")
                    phrase_count += 1

            except json.JSONDecodeError as e:
                print(f"  ❌ Invalid JSON {jf.name}: {e}")
                continue

        if not phrases:
            print("  ⚠ No valid phrases for this folder")
            continue

        out_path = out_root / f"{folder.name}_figures.txt"
        out_path.write_text("".join(phrases), encoding="utf-8")
        print(f"  ✅ Saved {len(phrases)//3} figures to {out_path}")

    print(f"\n✔ Done!")
    print(f"   Folders processed: {folder_count}")
    print(f"   Total phrases:     {phrase_count}")
    print(f"   Output folder:     {out_root}")

if __name__ == "__main__":
    main()


JSON root: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tomakepdf
TEXT root: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\tex_plain_text_files
Output:    C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\figure_phrases_by_json_folder

📁 JSON subfolder: 00011_arXiv_2310.10451
  📊 2310.10451_fig_3: arxiv=2310.10451, figure=3
  ✅ Direct match: 2310.10451.txt
  ✅ Saved 1 figures to C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\figure_phrases_by_json_folder\00011_arXiv_2310.10451_figures.txt

📁 JSON subfolder: 00017_arXiv_2402.04221
  📊 2402.04221_fig_16: arxiv=2402.04221, figure=16
  ✅ Direct match: 2402.04221.txt
  ❌ Figure 16 not found (total: 15)
  📊 2402.04221_fig_17: arxiv=2402.04221, figure=17
  ❌ Figure 17 not found (total: 15)
  📊 2402.04221_fig_2: arxiv=2402.04221, figure=2
  📊 2402.04221_fig_3: arxiv=2402.04221, figure=3
  ✅ Saved 2 figures to 

In [ ]:
"""
================================================================================
LaTeX Figure Name Extractor - Pipeline Step 8
================================================================================

Purpose: Extracts \includegraphics{} and \include{} filenames from LaTeX figure blocks

PIPELINE WORKFLOW:
1-7. Previous steps → figure_phrases_by_json_folder/
8. THIS SCRIPT → extracted_figure_names/ + all_extracted_names.csv


REQUIREMENTS:
No extra packages needed

WHAT IT EXTRACTS:
✅ \\includegraphics{figure1.pdf} → "figure1"
✅ \\include{circuit} → "circuit"  
✅ % \\includegraphics{img.pdf} → "img" (comments too)
✅ Figures/subplot.png → "subplot"
✅ Keeps .eps (non-standard) intact

INPUT:
figure_phrases_by_json_folder/
└── 00001_arXiv_2301.12345_figures.txt

OUTPUT:
extracted_figure_names/
├── 00001_arXiv_2301.12345_names.txt  ← "figure1", "circuit2", ...
└── ...

all_extracted_names.csv              ← Complete report
================================================================================
"""
import re
import csv
from pathlib import Path
import os

# ✅ FIXED: All paths are Path objects
INPUT_DIR = Path("figure_phrases_by_json_folder")
OUTPUT_DIR = Path("extracted_figure_names")
OUTPUT_CSV = Path("all_extracted_names.csv")

OUTPUT_DIR.mkdir(exist_ok=True)

# ... rest of the code unchanged ...


# MASTER REGEX - handles ALL your cases perfectly
# Matches \includegraphics{...} OR \include{...} and extracts basename
MASTER_PATTERN = re.compile(r'\\(?:include|includegraphics)(?:\[[^\]]*\])?\s*\{([^}]+?)(?:\.(?:pdf|png|eps|jpg|jpeg))?\}', re.IGNORECASE)

# Also handles commented out lines
COMMENTED_PATTERN = re.compile(r'%.*?\\(?:include|includegraphics)(?:\[[^\]]*\])?\s*\{([^}]+?)(?:\.(?:pdf|png|eps|jpg|jpeg))?\}', re.IGNORECASE)

def extract_figure_name(match: str) -> str:
    """
    Clean filename: remove path, keep basename (with/without extension)
    Examples:
    "img/circuit" → "circuit"
    "U_density_jmax.pdf" → "U_density_jmax"
    "Figures/Figure_2.pdf" → "Figure_2"
    "QtChnl.eps" → "QtChnl.eps" (keep .eps since not pdf/png)
    "SB+SY" → "SB+SY"
    """
    # Remove path prefix (everything before last /)
    basename = match.split('/')[-1].split('\\')[-1]
    
    # If it has .pdf/.png/.jpg/.jpeg → remove extension
    # Keep other extensions (.eps) intact
    if basename.lower().endswith(('.pdf', '.png', '.jpg', '.jpeg')):
        basename = basename[:-4]
    
    return basename.strip()

def process_file(txt_path: Path):
    """Process one text file and save extracted names."""
    content = txt_path.read_text(encoding="utf-8", errors="ignore")
    
    # Find all matches (active + commented)
    active_matches = MASTER_PATTERN.findall(content)
    comment_matches = COMMENTED_PATTERN.findall(content)
    
    all_raw_names = active_matches + comment_matches
    extracted_names = []
    
    for raw in all_raw_names:
        clean_name = extract_figure_name(raw)
        if clean_name and len(clean_name) > 0:  # skip empty
            extracted_names.append(clean_name)
    
    # Remove duplicates while preserving order
    unique_names = list(dict.fromkeys(extracted_names))
    
    # Save to text file (one name per line)
    output_txt = OUTPUT_DIR / f"{txt_path.stem}_names.txt"
    with open(output_txt, 'w', encoding='utf-8') as f:
        for name in unique_names:
            f.write(f"{name}\n")
    
    return [
        {'source_file': txt_path.name, 'raw_match': raw, 'clean_name': extract_figure_name(raw)}
        for raw in all_raw_names
    ]

def main():
    all_results = []
    
    for txt_file in Path(INPUT_DIR).glob("*.txt"):
        print(f"Processing {txt_file.name}...")
        results = process_file(txt_file)
        all_results.extend(results)
        print(f"  → Found {len(results)} figure references")
        print(f"  → Saved to {OUTPUT_DIR}/{txt_file.stem}_names.txt")
    
    # Save comprehensive CSV
    with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['source_file', 'raw_match', 'clean_name'])
        writer.writeheader()
        writer.writerows(all_results)
    
    print(f"\n✅ COMPLETE!")
    print(f"📁 Text files: {OUTPUT_DIR}")
    print(f"📊 CSV report: {OUTPUT_CSV}")
    print(f"📈 Total figure names extracted: {len(all_results)}")

if __name__ == "__main__":
    main()


Processing 00011_arXiv_2310.10451_figures.txt...
  → Found 1 figure references
  → Saved to C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figure_names/00011_arXiv_2310.10451_figures_names.txt
Processing 00017_arXiv_2402.04221_figures.txt...
  → Found 0 figure references
  → Saved to C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figure_names/00017_arXiv_2402.04221_figures_names.txt
Processing 00032_arXiv_2403.06340_figures.txt...
  → Found 2 figure references
  → Saved to C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figure_names/00032_arXiv_2403.06340_figures_names.txt
Processing 00034_arXiv_2506.18577_figures.txt...
  → Found 1 figure references
  → Saved to C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_figure_names/00034_arXiv_2506.18577_figures_names.txt
Processing 00040_arXiv_2403.19068_figures.txt...
  →

In [44]:
!pip install PyMuPDF


In [ ]:
"""
================================================================================
Archive Image Extractor - Pipeline Step 9
================================================================================

Purpose: Extracts actual figure images from arXiv source archives using figure names

PIPELINE WORKFLOW:
1-8. Previous steps → extracted_figure_names/
9. THIS SCRIPT → extracted_images/ (PNG images)


REQUIREMENTS:
pip install PyMuPDF  # pip install pymupdf (for PDF→PNG)

WHAT IT DOES:
1. Reads figure names from *_names.txt files
2. Finds matching .tar.gz/.zip archive for each paper
3. Searches archive for matching filenames
4. Extracts PNG directly, converts PDF→PNG
5. Organizes by paper: extracted_images/2301.12345_figures/

INPUT:
extracted_figure_names/     # Figure names from Step 8
├── 00001_arXiv_2310.10451_names.txt

arxiv_tex_sources/          # Archives from Step 4
├── 2310.10451.tar.gz

OUTPUT:
extracted_images/
└── 2310.10451_00001_arXiv_2310.10451/
    ├── circuit1.png
    ├── Figure_2.png
    └── subplot_a.png
================================================================================
"""

import os
import zipfile
import tarfile
import re
from pathlib import Path
import fitz  # pip install PyMuPDF

NAMES_DIR = Path("extracted_figure_names")
ZIP_DIR = Path("arxiv_tex_sources")
OUTPUT_IMAGES_DIR = Path("extracted_images")

OUTPUT_IMAGES_DIR.mkdir(exist_ok=True)

def extract_arxiv_id_from_filename(filename: str) -> str:
    """Extract arXiv ID from filename like '00011_arXiv_2310.10451_figures_names.txt' → '2310.10451'"""
    match = re.search(r'_arXiv_([0-9]+\.[0-9]+)', filename)
    return match.group(1) if match else None

def find_archive_for_arxiv(arxiv_id: str, archive_dir: Path) -> Path | None:
    """Find .tar, .tar.gz, .zip, .tgz matching arxiv_id"""
    patterns = [
        f"{arxiv_id}.tar",
        f"{arxiv_id}.tar.gz", 
        f"{arxiv_id}.tgz",
        f"{arxiv_id}.zip"
    ]
    
    for pattern in patterns:
        for archive_path in archive_dir.glob(pattern):
            return archive_path
    return None

def get_archive_namelist(archive_path: Path):
    """Get list of all files/folders inside archive (works for zip/tar/tar.gz)"""
    namelist = []
    
    if archive_path.suffix.lower() == '.zip':
        with zipfile.ZipFile(archive_path, 'r') as zf:
            namelist = zf.namelist()
    else:  # tar, tar.gz, tgz
        with tarfile.open(archive_path, 'r:*') as tf:
            namelist = tf.getnames()
    
    return namelist

def search_archive_for_figure(archive_path: Path, figure_name: str) -> list[tuple[str, str]]:
    """Search inside archive for folders/files matching figure_name"""
    matches = []
    namelist = get_archive_namelist(archive_path)
    
    for name in namelist:
        if figure_name.lower() in name.lower():
            matches.append((name, Path(name).name))  # (relative_path, filename)
    
    return matches

def extract_from_archive(archive_path: Path, rel_path: str, output_path: Path):
    """Extract file from archive and save (handles zip/tar/tar.gz)"""
    if archive_path.suffix.lower() == '.zip':
        with zipfile.ZipFile(archive_path, 'r') as zf:
            file_bytes = zf.read(rel_path)
    else:  # tar, tar.gz, tgz
        with tarfile.open(archive_path, 'r:*') as tf:
            file_obj = tf.extractfile(rel_path)
            if file_obj:
                file_bytes = file_obj.read()
            else:
                raise ValueError(f"Could not extract {rel_path}")
    
    return file_bytes

def convert_pdf_to_png(pdf_bytes: bytes, output_path: Path):
    """Convert PDF bytes to PNG"""
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    page = doc.load_page(0)
    pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
    pix.save(output_path)
    doc.close()

def main():
    processed_count = 0
    
    for names_file in NAMES_DIR.glob("*_names.txt"):
        print(f"\n📄 Processing: {names_file.name}")
        
        # Extract arXiv ID
        arxiv_id = extract_arxiv_id_from_filename(names_file.name)
        if not arxiv_id:
            print(f"  ❌ Could not extract arXiv ID from {names_file.name}")
            continue
        
        print(f"  🔍 arXiv ID: {arxiv_id}")
        
        # Find matching archive (.tar, .tar.gz, .zip, .tgz)
        archive_path = find_archive_for_arxiv(arxiv_id, ZIP_DIR)
        if not archive_path:
            print(f"  ❌ No archive found for {arxiv_id} (*.tar, *.tar.gz, *.zip, *.tgz)")
            continue
        
        print(f"  📦 Found archive: {archive_path.name}")
        
        # Create output subfolder
        paper_dir = OUTPUT_IMAGES_DIR / f"{arxiv_id}_{names_file.stem}"
        paper_dir.mkdir(exist_ok=True)
        
        # Read figure names (skip comments)
        figure_names = []
        with open(names_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line.startswith('#') and line:
                    figure_names.append(line)
        
        print(f"  🎯 Searching for {len(figure_names)} figure names...")
        
        found_count = 0
        namelist = get_archive_namelist(archive_path)
        
        for figure_name in figure_names:
            matches = search_archive_for_figure(archive_path, figure_name)
            
            for rel_path, filename in matches:
                # Create output filename
                ext = Path(filename).suffix or '.png'
                output_filename = f"{figure_name}{ext}"
                output_path = paper_dir / output_filename
                
                try:
                    file_bytes = extract_from_archive(archive_path, rel_path, output_path)
                    
                    # PNG: save directly
                    if filename.lower().endswith('.png'):
                        output_path.write_bytes(file_bytes)
                        print(f"  ✅ PNG: {output_filename}")
                    
                    # PDF: convert to PNG
                    elif filename.lower().endswith('.pdf'):
                        png_path = output_path.with_suffix('.png')
                        convert_pdf_to_png(file_bytes, png_path)
                        print(f"  ✅ PDF→PNG: {png_path.name}")
                    
                    # Other images: save directly
                    else:
                        output_path.write_bytes(file_bytes)
                        print(f"  ✅ Image: {output_filename}")
                    
                    found_count += 1
                    
                except Exception as e:
                    print(f"  ❌ Error {filename}: {e}")
        
        processed_count += 1
        print(f"  📊 Found {found_count} images for {arxiv_id}")
    
    print(f"\n🎉 COMPLETE!")
    print(f"📁 Images: {OUTPUT_IMAGES_DIR}")
    print(f"📊 Processed: {processed_count} files")

if __name__ == "__main__":
    main()



📄 Processing: 00011_arXiv_2310.10451_figures_names.txt
  🔍 arXiv ID: 2310.10451
  📦 Found archive: 2310.10451.tar.gz
  🎯 Searching for 1 figure names...
  ✅ PDF→PNG: circuit.png
  📊 Found 1 images for 2310.10451

📄 Processing: 00017_arXiv_2402.04221_figures_names.txt
  🔍 arXiv ID: 2402.04221
  📦 Found archive: 2402.04221.tar.gz
  🎯 Searching for 0 figure names...
  📊 Found 0 images for 2402.04221

📄 Processing: 00032_arXiv_2403.06340_figures_names.txt
  🔍 arXiv ID: 2403.06340
  📦 Found archive: 2403.06340.tar.gz
  🎯 Searching for 1 figure names...
  📊 Found 0 images for 2403.06340

📄 Processing: 00034_arXiv_2506.18577_figures_names.txt
  🔍 arXiv ID: 2506.18577
  📦 Found archive: 2506.18577.tar.gz
  🎯 Searching for 1 figure names...
  ✅ Image: QtChnl.eps
  📊 Found 1 images for 2506.18577

📄 Processing: 00040_arXiv_2403.19068_figures_names.txt
  🔍 arXiv ID: 2403.19068
  📦 Found archive: 2403.19068.tar.gz
  🎯 Searching for 4 figure names...
  ✅ PNG: LU.png
  ✅ PNG: SB+SY.png
  ✅ PNG: SB+

In [ ]:
"""
================================================================================
Quantum Circuit Image Dataset Analyzer - Pipeline Step 10
================================================================================

Purpose: Comprehensive analysis of extracted quantum circuit images

PIPELINE WORKFLOW:
1-9. Previous steps → extracted_images/ (FINAL DATASET)
10. THIS SCRIPT → Detailed statistics + validation report

REQUIREMENTS:
No extra packages needed

WHAT IT ANALYZES:
📊 Total images by paper
📈 Images by format (PNG, PDF, JPG, etc.)
✅ Dataset completeness check
📁 Folder structure validation

INPUT:
extracted_images/
└── 2310.10451_00001_arXiv_2310.10451/
    ├── circuit1.png
    ├── Figure_2.png
    └── ...

OUTPUT: Detailed console report + summary file
================================================================================
"""

from pathlib import Path
import os

IMAGES_DIR = Path("extracted_images")

def count_images(directory: Path):
    """Count all image files recursively (PNG, JPG, PDF, etc.)"""
    image_extensions = {'.png', '.jpg', '.jpeg', '.pdf', '.eps', '.gif', '.bmp', '.tiff'}
    total = 0
    
    print(f"📁 Counting images in: {directory}")
    print("-" * 60)
    
    for subdir in directory.iterdir():
        if subdir.is_dir():
            sub_count = 0
            print(f"\n📂 {subdir.name}/")
            
            for file in subdir.rglob("*"):
                if file.suffix.lower() in image_extensions:
                    sub_count += 1
                    total += 1
            
            print(f"   {sub_count:3d} images")
    
    print("-" * 60)
    print(f"🎯 TOTAL IMAGES: {total}")
    return total

def detailed_report(directory: Path):
    """Detailed report by paper and format"""
    image_extensions = {'.png', '.jpg', '.jpeg', '.pdf', '.eps', '.gif', '.bmp', '.tiff'}
    format_counts = {ext: 0 for ext in image_extensions}
    paper_counts = {}
    
    print(f"📊 DETAILED REPORT: {directory}")
    print("=" * 80)
    
    for subdir in directory.iterdir():
        if subdir.is_dir():
            paper_name = subdir.name
            paper_count = 0
            paper_formats = {}
            
            for file in subdir.rglob("*"):
                if file.suffix.lower() in image_extensions:
                    ext = file.suffix.lower()
                    format_counts[ext] = format_counts.get(ext, 0) + 1
                    paper_formats[ext] = paper_formats.get(ext, 0) + 1
                    paper_count += 1
            
            paper_counts[paper_name] = paper_count
            print(f"{paper_name:40s} | {paper_count:3d} images")
    
    print("=" * 80)
    print("📈 SUMMARY BY FORMAT:")
    for ext, count in format_counts.items():
        if count > 0:
            print(f"   {ext.upper():>4s}: {count:4d}")
    
    total_images = sum(paper_counts.values())
    print(f"\n🎉 GRAND TOTAL: {total_images} images across {len(paper_counts)} papers")

if __name__ == "__main__":
    if not IMAGES_DIR.exists():
        print(f"❌ Directory not found: {IMAGES_DIR}")
    else:
        count_images(IMAGES_DIR)
        print("\n" + "="*80 + "\n")
        detailed_report(IMAGES_DIR)


📁 Counting images in: C:\Users\Somantha\OneDrive - University of Sri Jayewardenepura\Desktop\Finalized\extracted_images
------------------------------------------------------------

📂 2310.10451_00011_arXiv_2310.10451_figures_names/
     1 images

📂 2401.01392_00632_arXiv_2401.01392_figures_names/
     3 images

📂 2401.08762_00354_arXiv_2401.08762_figures_names/
     1 images

📂 2401.14993_00192_arXiv_2401.14993_figures_names/
     4 images

📂 2402.04221_00017_arXiv_2402.04221_figures_names/
     0 images

📂 2402.13958_00589_arXiv_2402.13958_figures_names/
    10 images

📂 2402.17911_00938_arXiv_2402.17911_figures_names/
     1 images

📂 2402.19293_00576_arXiv_2402.19293_figures_names/
     2 images

📂 2403.06340_00032_arXiv_2403.06340_figures_names/
     0 images

📂 2403.14416_00513_arXiv_2403.14416_figures_names/
     1 images

📂 2403.19068_00040_arXiv_2403.19068_figures_names/
     4 images

📂 2404.16129_00321_arXiv_2404.16129_figures_names/
     0 images

📂 2404.17560_00328_arXiv_2